In [1]:
def Run_MNIST_Exp(Num_Epochs):
    %matplotlib notebook
    import keras as keras
    from keras.models import Sequential
    from keras.layers import LSTM, Dense,BatchNormalization,Dropout,Flatten, Conv1D, Conv2D,MaxPool2D
    from keras.losses import binary_crossentropy, categorical_crossentropy
    from keras.metrics import categorical_accuracy
    from keras import regularizers,optimizers
    from keras.regularizers import l2
    import numpy as np
    import scipy.io
    import matplotlib.pyplot as plt
    import gzip
    from keras.layers.core import Lambda
    from scipy.integrate import trapz
    import seaborn as sns

    from keras.layers.core import Lambda
    from keras import backend as K    


    def mini_batches(InputSample,BatchSize):
        Index = np.array(range(InputSample.shape[0]),dtype=int)
        NumBatches = np.int(InputSample.shape[0]/BatchSize)
        Removed = np.array([],dtype=int)

        BatchInd =[]
        for BatchLoop in range(NumBatches):
            RemainIndex = np.delete(Index,Removed)
            SampleInd = np.random.choice(RemainIndex,size=BatchSize,replace=False)
            Removed = np.append(Removed,SampleInd,axis=0)

            BatchInd.append(SampleInd)
        RemainIndex = np.delete(Index,Removed)
        BatchInd.append(RemainIndex)

        return BatchInd,NumBatches

    def Get_Feats_and_Targets(filename):
        import numpy as np

        def line_to_Feats(line):
            line = line.split(' ')
            Feats = np.asarray(line[0:1024])
            Target = np.zeros([3])
            Target[int(line[1024])] = 1
            return Feats,Target

        f = open(filename, 'r')
        lines = f.readlines()
        Features = []
        Targets = []
        for i in range(len(lines)-1):
            line = lines[i+1]
            Feats,Tgts = line_to_Feats(line)
            Features.append(Feats)
            Targets.append(Tgts)

        return np.asarray(Features,dtype = 'float64'), np.asarray(Targets,dtype = 'int')

    
    ConvFlag = 1
    
    FullInputs = scipy.io.loadmat('MNIST_TrainInputs.mat')
    FullInputs = FullInputs['images']

    FullTargets = scipy.io.loadmat('MNIST_TrainTargets.mat')
    FullTargets = FullTargets['targets']

    Validation_Cutoff = 0.5

    Validation_Cutoff = np.int(Validation_Cutoff*FullInputs.shape[0])

    ValInputs = FullInputs[Validation_Cutoff:,:]
    ValTargets = FullTargets[Validation_Cutoff:,:]

    TrainInputs = FullInputs[0:Validation_Cutoff,:]
    TrainTargets = FullTargets[0:Validation_Cutoff,:]
    
    if ConvFlag == 1:
        TrainInputs = TrainInputs.reshape([TrainInputs.shape[0],28,28,1])
        TrainInputs = np.swapaxes(TrainInputs,1,2)

        ValInputs = ValInputs.reshape([ValInputs.shape[0],28,28,1])
        ValInputs = np.swapaxes(ValInputs,1,2)
        
        data_dim2 = TrainInputs.shape[2]

    data_dim = TrainInputs.shape[1]
    NumSamples = TrainInputs.shape[0]
    Num_Targets = TrainTargets.shape[-1]

    index = np.linspace(0,NumSamples,NumSamples,endpoint=False,dtype=int)

    reg_coeff = 0.001

#     def Gen_Model(reg_coeff):
#         model = Sequential()
#         model.add(Dense(100,activation='tanh',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
#         model.add(Dense(100,activation='tanh',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
#         model.add(Dense(100,activation='tanh',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
#         model.add(Dense(Num_Targets,activation = 'softmax',kernel_regularizer=l2(reg_coeff),input_shape =(data_dim,)))
#         optim = optimizers.sgd(lr=0.001)
#         model.compile(loss='categorical_crossentropy',optimizer=optim,metrics=['categorical_accuracy'])
#         return model
    
    
    def Gen_Model(reg_coeff):
        model = Sequential()
        model.add(Conv2D(100,5,activation='relu',input_shape =(data_dim,data_dim2,1)))
        model.add(MaxPool2D())
        model.add(Conv2D(100,5,activation='relu'))
        model.add(MaxPool2D())
        model.add(Flatten())
        model.add(Dense(100,activation='relu',kernel_regularizer=l2(reg_coeff)))
        model.add(Dense(Num_Targets,activation = 'softmax',kernel_regularizer=l2(reg_coeff),input_shape =(data_dim,)))
        optim = optimizers.sgd(lr=0.001)
        model.compile(loss='categorical_crossentropy',optimizer=optim,metrics=['categorical_accuracy'])

        return model

    ISHard_model = Gen_Model(reg_coeff)

    ISEasy_model = Gen_Model(reg_coeff)

    Uni_model = Gen_Model(reg_coeff)

    ISHard_model.set_weights(Uni_model.get_weights())
    ISEasy_model.set_weights(Uni_model.get_weights())

    def average_dist_to_threshold(model,samples,Num_Targets):
        Output = model.predict(samples)
        Output -= 1/float(Num_Targets)
        Dist_to_Threshold = np.mean(np.abs(Output),1)
        return Dist_to_Threshold

    def acquisition_function_dist_to_threshold(model,samples,Num_Targets,temperature=1,Rescale=0):
        Output = model.predict(samples)
        Output -= 1/float(Num_Targets)
        Dist_to_Threshold = np.sum(np.abs(Output),1)
        Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
        Exp_Dist_to_Threshold *= 1
    #     Exp_Dist_to_Threshold = Dist_to_Threshold
        Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
        if Rescale == 1:
            Median_Prob = np.median(Sampling_Prob)
            Min_Prob = np.min(Sampling_Prob)
            Sampling_Prob[Sampling_Prob > Median_Prob] += Min_Prob/2
            Sampling_Prob[Sampling_Prob < Median_Prob] -= Min_Prob/2
        return Sampling_Prob

    count = 0 
    Num_Epochs = Num_Epochs
    Num_BurnIn = 1
    Batch_Size = 100

    Smoothing_Constant = 0

    Val_Error = np.zeros([Num_Epochs,3])
    Val_Acc = np.zeros([Num_Epochs,3])

    def Biased_Batch(Inputs,Targets,Batch_Size,Sampling_Probability,Weighting_Flag = 1):
        while True: 
            index = np.linspace(0,Inputs.shape[0],Inputs.shape[0],endpoint=False,dtype=int)
    #         Sampling_Probability = Sampling_Probability/Sampling_Probability.sum()
            Batch = np.random.choice(index,size=Batch_Size,replace=False,p=Sampling_Probability)
            Batch_Inputs = Inputs[Batch,:,:]
            Batch_Targets = Targets[Batch,:]
            if Weighting_Flag == 1:
                FullWeights = Sampling_Probability**-1
                FullWeights = FullWeights/np.mean(FullWeights)
            elif Weighting_Flag == -1:
                FullWeights = Sampling_Probability/np.mean(Sampling_Probability)
            else:
                FullWeights = np.ones(Inputs.shape[0])

            Weights = FullWeights[Batch]
            yield (Batch_Inputs,Batch_Targets,Weights)


    for EpochLoop in  range(Num_Epochs):
        if count < Num_BurnIn:
    #         ISHard_model.fit(TrainInputs,TrainTargets,batch_size = Batch_Size,epochs=1,validation_data=[ValInputs,ValTargets],verbose=0)
    #         Error = ISHard_model.test_on_batch(ValInputs,ValTargets)
    #         Val_Error[count,0] = Error[0]
    #         Val_Acc[count,0] = Error[1]

    #         ISEasy_model.fit(TrainInputs,TrainTargets,batch_size = Batch_Size,epochs=1,verbose=0)
    #         Error = ISEasy_model.test_on_batch(ValInputs,ValTargets)
    #         Val_Error[count,1] = Error[0]
    #         Val_Acc[count,1] = Error[1]

            Uni_model.fit(TrainInputs,TrainTargets,batch_size = Batch_Size,epochs=1,verbose=1)
            Error = Uni_model.test_on_batch(ValInputs,ValTargets)
            Val_Error[count,2] = Error[0]
            Val_Acc[count,2] = Error[1]

            ISHard_model.set_weights(Uni_model.get_weights())
            ISEasy_model.set_weights(Uni_model.get_weights())

            Error = ISHard_model.test_on_batch(ValInputs,ValTargets)
            Val_Error[count,0] = Error[0]
            Val_Acc[count,0] = Error[1]

            Error = ISEasy_model.test_on_batch(ValInputs,ValTargets)
            Val_Error[count,1] = Error[0]
            Val_Acc[count,1] = Error[1]


        else:
            Hard_Sampling_Probability = acquisition_function_dist_to_threshold(ISHard_model,TrainInputs,Num_Targets,0.6)
            Easy_SamplingProbability = 1/acquisition_function_dist_to_threshold(ISEasy_model,TrainInputs,Num_Targets,0.6)
            Easy_SamplingProbability/= Easy_SamplingProbability.sum().astype(float)
#             print(Hard_Sampling_Probability.max()/Hard_Sampling_Probability.min())

            MaxProbInd = Hard_Sampling_Probability.argmax()
            MinProbInd = Hard_Sampling_Probability.argmin()

            ISHard_model.fit_generator(Biased_Batch(TrainInputs,TrainTargets,Batch_Size,Hard_Sampling_Probability,0),
                                       steps_per_epoch=np.int(NumSamples/Batch_Size),verbose=0)

            ISEasy_model.fit_generator(Biased_Batch(TrainInputs,TrainTargets,Batch_Size,Easy_SamplingProbability,0),
                                     steps_per_epoch=np.int(NumSamples/Batch_Size),verbose=0)


            Uni_model.fit_generator(Biased_Batch(TrainInputs,TrainTargets,Batch_Size,1/np.float(NumSamples)*np.ones(NumSamples)),
                                    steps_per_epoch=np.int(NumSamples/Batch_Size),verbose=0)

            Error = ISHard_model.test_on_batch(ValInputs,ValTargets)
            Val_Error[count,0] = Error[0]
            Val_Acc[count,0] = Error[1]

            Error = ISEasy_model.test_on_batch(ValInputs,ValTargets)
            Val_Error[count,1] = Error[0]
            Val_Acc[count,1] = Error[1]

            Error = Uni_model.test_on_batch(ValInputs,ValTargets)
            Val_Error[count,2] = Error[0]
            Val_Acc[count,2] = Error[1]
            
            print(EpochLoop)

        count += 1

    TestInputs = scipy.io.loadmat('MNIST_TestInputs.mat')
    TestInputs = TestInputs['test_images']
    TestTargets = scipy.io.loadmat('MNIST_TestTargets.mat')
    TestTargets = TestTargets['test_targets']
    
    TestInputs = TestInputs.reshape([TestInputs.shape[0],28,28,1])
    TestInputs = np.swapaxes(TestInputs,1,2)

    TestError = np.zeros([3,2])
    TestError[0,:] = ISHard_model.test_on_batch(TestInputs,TestTargets)
    TestError[1,:] = ISEasy_model.test_on_batch(TestInputs,TestTargets)
    TestError[2,:] = Uni_model.test_on_batch(TestInputs,TestTargets)
    
    

    return TestError, Val_Error,Val_Acc

In [2]:
import numpy as np
import datetime
NumRuns = 1
NumEpochs = 50

TestError = np.zeros([3,2,NumRuns])
Val_Error = np.zeros([NumEpochs,3,NumRuns])
Val_Acc = np.zeros([NumEpochs,3,NumRuns])


In [3]:

for i in range(NumRuns):
    np.save("Test_Performance_MNIST_Conv",TestError)
    np.save("Val_Error_MNIST_Conv",Val_Error)
    np.save("Val_Acc_MNIST_Conv",Val_Acc)
    TestError[:,:,i],Val_Error[:,:,i],Val_Acc[:,:,i] = Run_MNIST_Exp(NumEpochs)
    print(i)


Using TensorFlow backend.


Epoch 1/1
30000/30000 [==============================] - 78s 3ms/step - loss: 2.4863 - categorical_accuracy: 0.1368
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
0


In [4]:
np.save("Test_Performance_MNIST_Conv",TestError)
np.save("Val_Error_MNIST_Conv",Val_Error)
np.save("Val_Acc_MNIST_Conv",Val_Acc)

In [8]:
Val_Error.shape

(1, 3)